## PID to CDM_ID File Renaming

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import shutil  # 파일 이동을 위해 shutil 모듈 사용

# 경로 설정
csv_path = r'C:\github\pt_no_person_id.csv'  ##병록번호와 cdm person_id 가 매칭된 csv 파일
file_dir = r'D:\extract'                    ## sig, hea, xml 파일이 존재하는 경로
rename_dir = r'D:\extract_cdm'              ## person_id 변환 완료된 파일들을 이동시킬 파일경로

# rename_dir 폴더가 없으면 생성
if not os.path.exists(rename_dir):
    os.makedirs(rename_dir)

# CSV 파일 로드 및 딕셔너리 생성
df = pd.read_csv(csv_path)
pt_no_to_person_id = df.set_index('pt_no')['person_id'].to_dict()

# 파일 리스트 가져오기
files = [f for f in os.listdir(file_dir) if '_' in f]

moved_count = 0

# 파일 이름 변경 및 이동
for file in tqdm(files, desc="Renaming and moving files"):
    try:
        # 파일명에서 마지막 '_' 다음에 있는 번호를 pt_no로 추출
        parts = file.rsplit('_', 1)
        base_name, pt_no_with_extension = parts[0], parts[1]
        
        pt_no, extension = pt_no_with_extension.split('.')  # pt_no와 확장자를 분리
        person_id = pt_no_to_person_id.get(int(pt_no))  # pt_no에 대응하는 person_id 찾기

        if person_id:
            # person_id를 사용해 새로운 파일명 생성
            new_filename = f'{base_name}_{int(person_id)}.{extension}'  

            old_file_path = os.path.join(file_dir, file)
            new_file_path = os.path.join(rename_dir, new_filename)
            
            # 파일 이동
            shutil.move(old_file_path, new_file_path)
            moved_count += 1  
        else:
            # 매칭되는 person_id가 없으면 파일을 원래 폴더에 그대로 둠
            continue

    except ValueError:
        print(f'Error processing file: {file}')

print(f'Task completed. A total of {moved_count} files were renamed and moved to {rename_dir}.')


Renaming and moving files: 100%|██████████| 2497/2497 [00:00<00:00, 416229.91it/s]

Task completed. A total of 0 files were renamed and moved to D:\extract_cdm.


## Update .hea Filename 

In [6]:
import os
from tqdm import tqdm

def replace_filename_in_hea_files(directory):
    converted_files_count = 0
    
    # Traverse the directory and find .hea files
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.hea'):
            file_path = os.path.join(directory, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                
                # Modify the line
                base_filename = filename.replace('.hea', '')
                lines[0] = lines[0].replace(lines[0].split()[0], base_filename)
                
                for i in range(1, len(lines)):
                    lines[i] = lines[i].replace(lines[i].split()[0], f"{base_filename}.SIG")
                
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.writelines(lines)
                
                converted_files_count += 1
            
            except Exception as e:
                print(f"Error processing file: {filename}. Error: {e}")
    
    print(f"Total number of converted files: {converted_files_count}")

directory_path = r'D:\extract_cdm'
replace_filename_in_hea_files(directory_path)


100%|██████████| 22135/22135 [32:38<00:00, 11.30it/s]  

Total number of converted files: 7348


##  XML 파일을 읽고, PID 값을 파일명에 있는 값으로 변경

In [10]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# 폴더 경로
folder_path = r'D:\extract_cdm'

# 폴더 내의 모든 XML 파일을 확인
xml_files = [file for file in os.listdir(folder_path) if file.endswith('.xml')]

# tqdm을 사용하여 진행 상황 표시
for file_name in tqdm(xml_files, desc="Processing XML files"):
    file_path = os.path.join(folder_path, file_name)
    
    try:
        # 파일명에서 '_' 뒤의 값을 추출
        new_pid = file_name.split('_')[-1].split('.')[0]
        
        # XML 파일 로드
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # PID 태그를 찾고 값을 새로운 PID로 변경
        for pid in root.iter('PID'):
            pid.text = new_pid
        
        # 변경된 XML을 파일에 다시 저장
        tree.write(file_path)
    
    except ET.ParseError:
        print(f"ParseError: Unable to process {file_name}. The file may be malformed.")
    except Exception as e:
        print(f"An error occurred while processing {file_name}: {e}")

print("모든 XML 파일의 PID 값이 변경되었습니다.")


Processing XML files:  95%|█████████▍| 7043/7438 [02:20<00:10, 36.87it/s]

ParseError: Unable to process 195_5_2222880.xml. The file may be malformed.


Processing XML files: 100%|██████████| 7438/7438 [04:33<00:00, 27.18it/s]

모든 XML 파일의 PID 값이 변경되었습니다.
